BYNmasivofy2024*

In [121]:
# Importa las librerias a usar
import pandas as pd
import re
import numpy as np


importar arvhivos a procesar

In [122]:
# ubicacion de los archivos
patch1 = '../input/Kilometros en Servicio MIO - 05-Mar-2024.xlsx'# Se importa el archivo xlxs
patch2 =  '../input/REPORTE FINALIZADO 05_MARZ_2024.xlsx'
patch3 = '../input/JERARQUIAS.xlsx'

In [123]:
# Se importa el archivo xlxs
kmsmio = pd.read_excel(patch1, sheet_name='Programacion')
repofi = pd.read_excel(patch2, sheet_name='Programacion')
bitacora = pd.read_excel(patch2, sheet_name='Bitacora')
jerarquias = pd.read_excel(patch3, sheet_name='Hoja1')
# Se eliminan las filas y las columnas vacias
kmsmio = kmsmio.dropna(axis=0, how='all').dropna(axis=1, how='all')
repofi = repofi.dropna(axis=0, how='all').dropna(axis=1, how='all')
bitacora = bitacora.dropna(axis=0, how='all').dropna(axis=1, how='all')

In [124]:
# Se crea el df de adicionales
adi = kmsmio[kmsmio['Kilometros Programados Plan'].isna()]

Porcesar K2

In [125]:
# Filtra el DF por Kms en Servicio
kmsmio = kmsmio[kmsmio['Tipo Kilómetros']== 'Kms en Servicio']
kmsmio = kmsmio[kmsmio['Kilometros Programados Plan'].notna()]

In [126]:
# Cambiamos la hora 24 por 0
kmsmio['Desde'] = kmsmio['Desde'].replace('24', '0')
kmsmio = kmsmio[kmsmio['Desde'] != '0']


In [127]:
# Se crea una nueva columna 'id'
kmsmio['id'] = kmsmio.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['Desde']), str(row['hasta'])]), axis=1)


In [128]:
# Se verifica el tamaño del df
kmsmio.shape

(18645, 40)

In [129]:
# Se verifica que la columna id no tenga id repetidos
kmsmio['id'].value_counts().sort_values(ascending=False)

id
A14A-1-05:00:00-05:12:10                              1
T47B-2-19:21:48-20:01:10                              1
A14A-1-05:26:00-05:38:10                              1
A14A-1-05:38:55-05:51:15                              1
A14A-1-05:52:00-06:04:43                              1
                                                     ..
A35A-99753-1900-01-01 00:03:04-1900-01-01 00:14:39    1
A35A-99753-23:03:04-23:14:39                          1
A35A-99753-21:33:04-21:44:07                          1
A35A-99753-21:30:02-21:30:17                          1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

Procesar reporte finalizado

In [130]:
# Filtra el DF por Kms en Servicio
repofi = repofi[repofi['Tipo de kilometros']== 'Kms en Servicio']
# Se crea una nueva columna 'id'
repofi['id'] = repofi.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                  str(row['desde']),str(row['hasta'])]), axis=1)


In [131]:
# Se verifica que el id no tenga datos repetidos
repofi['id'].value_counts().sort_values(ascending=False)

id
A14A-1-05:00:00-05:12:10                              1
T47B-2-19:21:48-20:01:10                              1
A14A-1-05:26:00-05:38:10                              1
A14A-1-05:38:55-05:51:15                              1
A14A-1-05:52:00-06:04:43                              1
                                                     ..
A35A-99753-1900-01-01 00:03:04-1900-01-01 00:14:39    1
A35A-99753-23:03:04-23:14:39                          1
A35A-99753-21:33:04-21:44:07                          1
A35A-99753-21:30:02-21:30:17                          1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

In [132]:
# Se verifica el tamaño del df
repofi.shape

(18645, 32)

In [133]:
reporf = repofi[['id', 'Largo', 'Número de secuencia de carro', 'Secuencia de arcos', 'Concesionario de Transporte']]

Se realiza el respectivo Merge con los dos df

In [134]:
# Se realiza el merge con outer en la columna id
merge = pd.merge(kmsmio, repofi, on='id', how='outer')

In [135]:
# Se verifica que el numero de filas en los df sea igual (True), para poder concluir que el merge fue exitoso
merge.shape[0] == kmsmio.shape[0] == repofi.shape[0]

True

Calificaciones de jerarquias en Bitacora

In [136]:
# Selecciona las columnas a usar
jerarquias = jerarquias[['JERARQUIA HGI RECO2',	'CALIFICACION']]

In [137]:
# Esto elimina los espacios en blanco adelante y atras
jerarquias['JERARQUIA HGI RECO2'] = jerarquias['JERARQUIA HGI RECO2'].str.strip()

In [138]:
#combina los el df de bitacora con el de jerarquias
bitacora = pd.merge(bitacora, jerarquias, left_on='Jerarquía', right_on='JERARQUIA HGI RECO2', how='left' )

In [139]:
# Esta linea es para verificar las filas que no se cruzaron por jerarquia y agregarlas a la matriz fe jererquia en input
x = bitacora[(bitacora['Jerarquía'].str.startswith('Salida'))& bitacora['CALIFICACION'].isnull()]
x['Jerarquía'].value_counts()

Series([], Name: count, dtype: int64)

In [140]:
# Este codigo elimina los duplicado de la columna calificaciones eliminando el que no afecta a kilometros

# Convertir la columna 'CALIFICACION' a numérica para poder comparar y encontrar el máximo
bitacora['CALIFICACION'] = pd.to_numeric(bitacora['CALIFICACION'], errors='coerce')

# Agrupar por las columnas 'Viaje' y 'Tarea', y mantener el valor máximo de 'CALIFICACION'
bitacora_grouped = bitacora.groupby(['Viaje', 'Tarea']).agg({'CALIFICACION': 'max'}).reset_index()

# Identificar las filas duplicadas en la tabla original
duplicados = bitacora[bitacora.duplicated(subset=['Viaje', 'Tarea'], keep=False)]

# Mantener solo una fila para cada grupo de duplicados en 'Viaje' y 'Tarea'
bitacora_sin_duplicados = bitacora.drop_duplicates(subset=['Viaje', 'Tarea'])

# Fusionar los datos filtrados con los valores máximos de 'CALIFICACION'
bitacora_final = pd.merge(bitacora_sin_duplicados, bitacora_grouped, on=['Viaje', 'Tarea'])

# Eliminar cualquiera de las filas duplicadas si ambas tienen valores nulos en 'CALIFICACION'
bitacora = bitacora_final.drop_duplicates(subset=['Viaje', 'Tarea'], keep='last', ignore_index=True)
    

In [141]:
# Se verifica que no hay duplicados
bitacora[bitacora.duplicated(subset=['Viaje', 'Tarea'], keep=False)]

,# Caso MC,Tipo novedad,Jerarquía,Tarea,Viaje,Parada,Operador,Línea,Vehículo,Hora,...,Jerarquia,Tipo de Novedad,Novedad,Retoma/Salida,Unnamed: 16,Unnamed: 17,Unnamed: 18,JERARQUIA HGI RECO2,CALIFICACION_x,CALIFICACION_y


In [142]:
# Se elimina la columna CALIFICACION_X y se renombra la columna CALIFICACION_Y
bitacora.drop(columns='CALIFICACION_x', axis=1, inplace=True)
bitacora.rename(columns={'CALIFICACION_y': 'CALIFICACION'}, inplace=True)


Merge con bitacora


In [143]:
# Se reemplan las filas nulas por ceros de las dos columnas para posteriormente carbiar el tipo de dato
bitacora[['Tarea', 'Viaje']] = bitacora[['Tarea', 'Viaje']].fillna(0).astype(int)

In [144]:
# Se selecionan las columnas de Bitacora que se van a usar
bitacora = bitacora[['Jerarquía',	'Tarea',	'Viaje', 'Parada', 'Vehículo', 'Novedad', 'CALIFICACION']]
# Se filtran los casos que no tiene numero de viaje ya que no tiene retoma
bitacora = bitacora[bitacora['Viaje'] != 0]

In [145]:
# Se compara las filas de los df, el resultado debe ser False
bitacora.shape[0]==bitacora.shape[0]

True

In [146]:
# Se agrega al merge anterior la informacion de bitacora
# Importante tener en cuenta los vehiculos que tengan mas de un caso en la bitacora, esto genera id duplicados.
mergebitacora = pd.merge(merge, bitacora, left_on=['Número de tarea vehículo_x', 'Número de viaje_x'], right_on=['Tarea', 'Viaje'], how='outer' )

In [147]:
# Revisar este resultado, ya que los valores que no coinceden deben ser iguales en el filtro
mergebitacora.shape[0]- merge.shape[0]

0

In [148]:
# Cambia Blanco y Negro Masivo por BYN en la columna Concesionario Programado
mergebitacora['Concesionario Programado'] = mergebitacora['Concesionario Programado'].replace('Blanco y Negro Masivo','BYN')

OK

In [149]:
# Se reemplazan los valores 0 por -1
#mergebitacora['Incumplidos_y'].replace(0, -1, inplace=True)
#mergebitacora['Incumplidos_x'].replace(0, -1, inplace=True)

In [150]:
# Se reemplazan los valores nulos por ceros
#mergebitacora['Incumplidos_y'].fillna(0, inplace=True)
#mergebitacora['Incumplidos_x'].fillna(0, inplace=True)

In [151]:
# Se reemplazan los valores 0 por -1
#mergebitacora['Incumplidos_y'].replace(0, -1, inplace=True)
#mergebitacora['Incumplidos_x'].replace(0, -1, inplace=True)
# Se reemplazan los valores nulos por ceros
#mergebitacora['Incumplidos_y'].fillna(0, inplace=True)
#mergebitacora['Incumplidos_x'].fillna(0, inplace=True)
# Se cambia el tipo de dato para las columnas 
#mergebitacora['VIAJE'] =  mergebitacora['VIAJE'].astype(int)
#mergebitacora['TAREA'] = mergebitacora['TAREA'].astype(int)


In [152]:
x = mergebitacora

In [153]:
y =x[(x['Incumplidos_x'].isna())&(x['Incumplidos_y'].notna())]

In [154]:
y[['Incumplidos_x','Incumplidos_y']]

,Incumplidos_x,Incumplidos_y
1112,NaN,2.0
3169,NaN,2.0
4069,NaN,2.0
4812,NaN,2.0
8868,NaN,2.0
9377,NaN,2.0
9794,NaN,2.0
9818,NaN,2.0
10032,NaN,2.0
10523,NaN,2.0


In [ ]:
def vehiculos_xy_texto():

In [ ]:
def vehiculo_xy(row):
    if pd.isna(row['Número de Vehículo']) and  pd.notnan(row['Número de secuencia de carro']): 
        return row['Número de secuencia de carro']
    elif pd.notna(row['Número de Vehículo']) and  pd.nan(row['Número de secuencia de carro']):
        return row['Número de Vehículo']
    elif row['Número de Vehículo'] !=  row['Número de secuencia de carro']:
        return row['Número de Vehículo']


In [155]:
def prueba(row, df):
    """
    Calcula el valor según las condiciones especificadas.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.
    df (pandas.DataFrame): DataFrame que contiene los datos de la bitácora.

    Retorna:
    int or float or np.nan: El valor calculado según las condiciones.
    """
    # Obtener el índice de la fila actual
    index = row.name
    
    # Calcular el índice de la fila siguiente
    next_index = index + 1
    next_index_next = index + 2
    
    # Obtener la fila siguiente si el índice es válido, de lo contrario, establecerla como None
    next_row = df.iloc[next_index] if next_index < len(df) else None
    next_row_next = df.iloc[next_index_next] if next_index_next < len(df) else None

    if row['Tipo Kilómetros'] == "Kms en Servicio" and pd.isna(row['Número de secuencia de carro']):
        if row['Tipo de viaje corto_x'] == "Viaje comercial":
            return 2
        elif row['Tipo de viaje corto_x'] in ["Viaje de punto operacional", "Viaje de posicionamiento"]:
            if next_row is not None and pd.isna(next_row['Número de secuencia de carro']):
                if (next_row['Incumplidos_y'] == 2) and (next_row['Largo_y'] in [0, 1]):
                    return 1
                elif (next_row['Largo_y'] not in [0, 1]) or (next_row['Largo_y']  in [0, 1]):
                    if pd.notna(row['Incumplidos_y']):
                        return next_row_next['Incumplidos_y'] 
                    else:
                        return 2
            else:
                if next_row is not None and next_row['Largo_y'] in [0, 1]:
                    return 1
                else:
                    return 2
        else:
            return np.nan
    elif (row['Número de secuencia de carro'] in ["CAMBIO", "CANCELADA"]) and row['Tipo Kilómetros'] == "Kms en Servicio":
        return 0


In [156]:
mergebitacora['prueba'] = np.nan

In [157]:
# Aplicar la función a todo el DataFrame
mergebitacora['prueba'] = mergebitacora.apply(prueba, axis=1, args=(mergebitacora,))

In [120]:
mergebitacora['prueba'].value_counts()

prueba
2.0    11770
0.0       38
Name: count, dtype: int64

In [158]:
def col_calificaciones(row, df):
    """
    Calcula el valor del incumplimiento según las condiciones especificadas.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.
    df (pandas.DataFrame): DataFrame que contiene los datos de la bitácora.

    Retorna:
    int or float or np.nan: El valor del incumplimiento calculado según las condiciones.
    """
    # Obtener el índice de la fila actual
    #index = row.name
    
    # Calcular el índice de la fila siguiente
    #next_index = index + 1
    #next_index_next = index +2
    # Obtener la fila siguiente si el índice es válido, de lo contrario, establecerla como None
    #next_row = df.iloc[next_index] if next_index < len(df) else None
    #next_row_next = df.iloc[next_index_next] if next_index_next < len(df) else None

    # Condiciones para calcular el valor del incumplimiento
    if pd.isna(row['Incumplidos_x']) and pd.notna(row['Incumplidos_y']):
        return np.nan
    elif row['Incumplidos_x'] == 3:
        return row['Incumplidos_x']
    elif row['Incumplidos_x'] < row['Incumplidos_y'] and row['Incumplidos_x'] != 2:
        return row['Incumplidos_x']
    elif pd.isna(row['Incumplidos_x']) and pd.notnull(row['Incumplidos_y']):
        return np.nan
    
    else:
        return np.nan
    
    



In [160]:
mergebitacora.to_excel('../output/incumplidos.xlsx', sheet_name='prueba')

In [159]:
# Aplicar la función a todo el DataFrame
mergebitacora['Incumplidos_y'] = mergebitacora.apply(col_calificaciones, axis=1, args=(mergebitacora,))

Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0
Valor de Incumplidos_x: nan
Valor de Incumplidos_y: 2.0


In [851]:
# Se reemplaza nuevamente el cero por NAN y el menos -1 por cero
#mergebitacora['Incumplidos_y'].replace(0, np.nan, inplace=True)
#mergebitacora['Incumplidos_x'].replace(0, np.nan, inplace=True)

#mergebitacora['Incumplidos_y'].replace(-1, 0, inplace=True)
#mergebitacora['Incumplidos_x'].replace(-1, 0, inplace=True)

verificacion

In [41]:
z = mergebitacora[(mergebitacora['Incumplidos_x'].isna())&(mergebitacora['Incumplidos_y'].notna())]

In [42]:
z[['VIAJE', 'TAREA', 'Incumplidos_x', 'Incumplidos_y']]

,VIAJE,TAREA,Incumplidos_x,Incumplidos_y


In [43]:
mergebitacora[['VIAJE', 'TAREA', 'Incumplidos_x', 'Incumplidos_y']]

,VIAJE,TAREA,Incumplidos_x,Incumplidos_y
0,1.0,1.0,NaN,NaN
1,2.0,1.0,NaN,NaN
2,9.0,1.0,NaN,NaN
3,10.0,1.0,NaN,NaN
4,17.0,1.0,NaN,NaN
...,...,...,...,...
18640,9955.0,99753.0,NaN,NaN
18641,9956.0,99753.0,NaN,NaN
18642,9957.0,99753.0,NaN,NaN
18643,9958.0,99753.0,NaN,NaN


verificacion

In [ ]:
##
##
##no va
##
##
# Definir una función para aplicar a cada grupo
def eliminar_duplicados_bitacora(row):
    # Verificar si hay al menos un valor en 'Incumplidos_x'
    if row['Incumplidos_x'].isnull() and row['Incumplidos_x'].isnull():   
        return row[row['Incumplidos_x'].notnull()]  # Conservar la fila con 'Incumplidos_x' no nulo
    # Verificar si hay al menos un valor en 'Incumplidos_y'
    elif row['Incumplidos_y'].notnull().any():
        return row[row['Incumplidos_y'].notnull()]  # Conservar la fila con 'Incumplidos_y' no nulo
    # Conservar la fila con el valor más bajo en 'CALIFICACION'
    else:
        return row.loc[row['CALIFICACION'].idxmin()]

In [ ]:
# Muestra los id duplicados producto del merge con bitacora
mergebitacora[mergebitacora['id'].duplicated()]

Revision de Adicionales

Columna condicional 

In [ ]:
# Se cambian los vehiculos del cierre por los de k2##
mergebitacora['Número de secuencia de carro'] = [x if pd.notna(x) else y for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]
# Marca en la columnas Cambios los cambios que realiza el código
mergebitacora['condicion'] = [ np.nan if  pd.notna(x) else 'Se Cambia # V RF' for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]


In [ ]:
# # Función para actualizar el texto de la columna condicion para Largo_y si Largo_x es mayor
def actualizar_largo_texto(row):
    if pd.isna(row['Largo_y']) and pd.isna(row['condicion']) :
        return 'Cambia largo por NAN'
    elif pd.isna(row['Largo_y']) and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia largo por NAN'
    elif row['Largo_x'] > row['Largo_y'] and pd.isna(row['condicion']):
        return 'Cambia x Mayor largo'
    elif row['Largo_x'] > row['Largo_y'] and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia x Mayor largo'
    else:
        if pd.notnull(row['condicion']):
            return row['condicion']
            

In [ ]:
# Función para actualizar los valores de Largo_y si Largo_x es mayor
def actualizar_largo(row):
    if pd.isna(row['Largo_y']):
        return row['Largo_x']
    elif row['Largo_x'] > row['Largo_y']:
        return row['Largo_x']
    else:
        return row['Largo_y']

In [ ]:
# Marcar las filas donde se realizan cambios en una nueva columna
mergebitacora['condicion'] = mergebitacora.apply(actualizar_largo_texto, axis=1)

In [ ]:
# Aplicar la función a la columna Largo_y
mergebitacora['Largo_y'] = mergebitacora.apply(actualizar_largo, axis=1)

In [ ]:
#Se verifican los cambios realizados
mergebitacora['condicion'].value_counts()

In [ ]:
'Número de secuencia de carro', 'Secuencia de arcos', 'Concesionario Programado'

In [ ]:
# Condiciones para la columna condicional
condicion1 = mergebitacora['Largo_x'] != mergebitacora['Largo_y']
condicion2 = mergebitacora['Número de Vehículo'] != mergebitacora['Número de secuencia de carro']
condicion3 = mergebitacora['Secuencia de arcos_x'] != mergebitacora['Secuencia de arcos_y']
condicion4 = mergebitacora['Concesionario de Transporte_x'] != mergebitacora['Concesionario Programado']

# Aplicar condiciones y asignar valores a la nueva columna
mergebitacora['condicional2'] = np.where(condicion1 & condicion2 & condicion3 & condicion4, 'Revisar largo, parada, vehículo y consecionario',
                                    np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                        np.where(condicion1 & condicion2 & condicion4, 'Revisar largo, parada y consecionario',         
                                            np.where(condicion1 & condicion3 & condicion4, 'Revisar largo, parada y consecionarios',
                                                np.where(condicion2 & condicion3 & condicion4, 'Revisar vehiculo, parada y consecionarios',        
                                                    np.where(condicion1 & condicion2, 'Revisar largo y vehiculo',
                                                        np.where(condicion1 & condicion3, 'Revisar largo y parada',
                                                            np.where(condicion1 & condicion4, 'Revisar largo y consecionario', 
                                                                np.where(condicion2 & condicion3, 'Revisar vehiculo y parada',
                                                                    np.where(condicion2 & condicion4, 'Revisar vehiculo y consecionario',        
                                                                        np.where(condicion3 & condicion4, 'Revisar parada y consecionario',                                            
                                                                            np.where(condicion1, 'Revisar largo', 
                                                                                np.where(condicion2, 'Revisar numero de vehiculo',
                                                                                    np.where(condicion3, 'Revisar paradas',
                                                                                        np.where(condicion4, 'Revisar consesionario', 'OK')))))))))))))))

In [ ]:
mergebitacora['condicional2'].value_counts()

In [ ]:
# Exporta a un archivo excel el Df
#mergebitacora.to_excel('../output/prueba_mergebita.xlsx', index=False, sheet_name='k2_procesado')